<a href="https://colab.research.google.com/github/mehdihosseinimoghadam/Routing/blob/main/OSRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install polyline geocoder -q

     |████████████████████████████████| 98 kB 2.7 MB/s 


# This Code Uses http://router.project-osrm.org/ server but if you need to handle great number of requests, it is better to use [This Docker file]( https://hub.docker.com/r/osrm/osrm-backend/) and do some load blancing


In [6]:
import logging
import requests
import json
import polyline
import folium
from folium.plugins import MeasureControl
import geocoder

from functools import lru_cache

logger = logging.getLogger(__name__)
DEBUG = True

@lru_cache(maxsize=None)
def geocode(location):
    return _geocode(location)

def _geocode(location):
    import geocoder
    g = geocoder.osm(location)
    return g.latlng

@lru_cache(maxsize=None)
def get_route(olat, olng, dlat, dlng):
    response = _get_route(olat, olng, dlat, dlng)
    return response

def _get_route(olat, olng, dlat, dlng):
    url = f'http://router.project-osrm.org/route/v1/driving/{olng},{olat};{dlng},{dlat}?steps=true&annotations=true&geometries=polyline&overview=full&annotations=speed&alternatives=true'
    # logger.debug(url)
    response = None

    try:
        logger.debug(f'====== OSRM: {url}')
        response = requests.get(url, verify=False)
    except Exception as ex:
        raise
    
    # logger.debug(response.text)
    if response and response.text:
        response_dict = json.loads(response.text)
        #possible = pd.DataFrame([{'Distance': (route['distance'] / 1000) *  0.621371 , route['weight_name']: route['weight']} for route in response_dict['routes']])
        return response_dict
    else:
        return None

def get_routing_map(origin, destination, zoom=5):
    orig_latlng = geocode(origin)
    dest_latlng = geocode(destination)

    resp = get_route(orig_latlng[0], orig_latlng[1], dest_latlng[0], dest_latlng[1])

    decoded = polyline.decode(resp["routes"][0]['geometry'])
    distance = resp["routes"][0]['distance'] * 0.000621371
    duration = resp["routes"][0]['duration'] / 60

    map2 = folium.Map(location=(orig_latlng[0], orig_latlng[1]), zoom_start=zoom,
                                    control_scale=True)
    # map2.add_child(MeasureControl(
    #     primary_length_unit='miles',
    #     secondary_length_unit='meters',
    #     primary_area_unit='acres',
    #     secondary_area_unit='sqmeters')
    # )

    folium.PolyLine(locations=decoded, color="blue").add_to(map2)

    print(f"{duration} minutes")
    print(f"{distance} miles")

    return map2, resp


In [7]:
map ,resp= get_routing_map('AB21 7EE', 'TR19 7EW', zoom=15)
map

768.705 minutes
699.2288484371 miles


In [8]:
import json

json_object = json.dumps(resp, indent=4)
 
# Writing to sample.json
with open("sample.json", "w") as outfile:
    outfile.write(json_object)

# For navigation instructions you can also use this script,
# if you are using Javascript [use this package](https://github.com/Project-OSRM/osrm-text-instructions)


In [10]:
def directions(resp):

      d = {1:'first' , 2:'second' , 3:'third' , 4:'fourth' }
      l = []

      for i in range(len(resp['routes'][0]['legs'][0]['steps'])-1):

          if i == 0:
            txt0 = 'Start, Have a good trip.'
            txt1 = 'In ' + str(round(resp['routes'][0]['legs'][0]['steps'][i]['distance'])) + ' meter '
            txt2 =  resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['type'] + ' ' + resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['modifier'] + ' onto ' + resp['routes'][0]['legs'][0]['steps'][i+1]['name']

            txt = txt0 + txt1 + txt2
            # print(txt)
            l.append(txt)

          elif resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['type'] == 'roundabout':
                
                txt0 = 'In ' + str(round(resp['routes'][0]['legs'][0]['steps'][i]['distance'])) + ' meter '
                txt1 = 'In the ' + resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['type'] + ' take the ' + d[resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['exit']] + ' exit  onto ' + resp['routes'][0]['legs'][0]['steps'][i+1]['name'] 
                txt = txt0 + txt1
                # print(txt)


          elif i == len(resp['routes'][0]['legs'][0]['steps']) - 2: 
            txt = 'You have arrived to your destination '
            l.append(txt)

          else:
                try:
                  txt0 = 'In ' + str(round(resp['routes'][0]['legs'][0]['steps'][i]['distance'])) + ' meter '
                  txt1 = resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['type'] + ' ' + resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['modifier'] + ' onto ' + resp['routes'][0]['legs'][0]['steps'][i+1]['name']
                  txt = txt0 + txt1
                  # print(txt)
                  l.append(txt)
                except:
                  txt0 = 'In ' + str(round(resp['routes'][0]['legs'][0]['steps'][i]['distance'])) + ' meter '
                  txt1 = resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['type'] + ' ' + resp['routes'][0]['legs'][0]['steps'][i+1]['maneuver']['modifier'] + ' onto ' + resp['routes'][0]['legs'][0]['steps'][i+1]['name']
                  txt = txt0 + txt1
                  # print(txt)
                  l.append(txt)

      return l    


In [11]:
directions(resp)

['Start, Have a good trip.In 28 meter turn left onto ',
 'In 42 meter turn left onto Avon Place',
 'In 59 meter turn left onto Victoria Street',
 'In 273 meter turn right onto Farburn Terrace',
 'In 5 meter exit roundabout straight onto ',
 'In 55 meter new name slight left onto Wellheads Drive',
 'In 2100 meter end of road right onto Dyce Drive',
 'In 703 meter turn left onto Argyll Road',
 'In 601 meter rotary straight onto Craibstone Junction Link',
 'In 155 meter exit rotary straight onto Craibstone Junction Link',
 'In 356 meter off ramp slight left onto ',
 'In 238 meter fork slight right onto ',
 'In 197 meter merge slight right onto Aberdeen Western Peripheral Route',
 'In 13035 meter turn right onto Aberdeen Western Peripheral Route',
 'In 11507 meter rotary left onto ',
 'In 75 meter exit rotary straight onto ',
 'In 230 meter turn right onto ',
 'In 681 meter merge slight right onto Stonehaven Bypass',
 'In 78035 meter rotary straight onto Forfar Road',
 'In 40 meter exit ro